In [1]:
import os
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName("solution_ex_06").getOrCreate()

22/05/18 14:27:37 WARN Utils: Your hostname, certusr-VirtualBox resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
22/05/18 14:27:37 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/05/18 14:27:40 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/05/18 14:27:43 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/05/18 14:27:43 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [3]:
inputPath = './data'
outputPath = './output_ex_05'

print(os.path.exists(inputPath))
print(os.path.exists(outputPath))

True
False


In [4]:
logRDD = spark.sparkContext.textFile(inputPath)

In [14]:
pm10 =  logRDD.map(lambda x: float(x.split(',')[2]))

# Select/compute the maximum PM10 value
# maxPM10Value = pm10ValuesRDD.reduce(lambda value1, value2: max(value1,value2))

maxPM10 = pm10.top(1)[0]
maxPM10

72.9

In [16]:
# Filter the content of logRDD
# Select only the line(s) associated with the maxPM10Value
selectedRecordsRDD = logRDD.filter(lambda PM10Reading: float(PM10Reading.split(',')[2])==maxPM10)

selectedRecordsRDD.take(5)

['s3,2016-01-03,72.9']

In [19]:
# Extract the dates from the selected records (second field of each string)
datesRDD = selectedRecordsRDD.map(lambda PM10Reading: PM10Reading.split(',')[1])

# Remove duplicates, if any
distinctDatesRDD = datesRDD.distinct()
distinctDatesRDD.take(5)

['2016-01-03']

In [20]:
# Store the result in the output folder
distinctDatesRDD.saveAsTextFile(outputPath)